## MISNT Data Classifier with Pytorch
Example of CNN project based class task

#### Content  
Code up MNIST Deep Neural Network Classifier from scratch with various choice of "Hyper parameters"

#### Work environment
Google Colaboratory  
How to use Colaboratory [Geeksforgeeks: How to use google colab](https://www.geeksforgeeks.org/how-to-use-google-colab/)

-------------
### Import google.colab module to mount your gdrive

In [21]:
# Amount your drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


-----------------
### Import necessary modules

In [0]:
import os
import argparse
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torchvision import datasets, transforms

### Set parameters as argument parser

In [0]:
def get_FLAGS(verbose=True):
  p = argparse.ArgumentParser()
  p.add_argument(
      '--batch_size',
      type=int,
      default=32          
  )
  p.add_argument(
      '--lr',
      type=float,
      default=0.001, 
      help='learning rate'
  )
  p.add_argument(
      '--num_epoch',
      type=int,
      default=5        
  )
  args = p.parse_args(args=[])
  # + Memo
  # in script type coding line, you should specify the above line as 
  # args = parser.parse_args()
  if verbose:
    for f in args.__dict__:
      print("\t", f, (25 - len(f)) * " ", args.__dict__[f])
  return args

In [24]:
FLAGS = get_FLAGS()

	 batch_size                 32
	 lr                         0.001
	 num_epoch                  5


### Load MNIST dataset

In [0]:
# train dataloader
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        root='./mnist_data',
        train=True,
        download=True,
        transform=transforms.Compose([
            transforms.Resize((28,28)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])
    ),
    batch_size=FLAGS.batch_size,
    shuffle=True,
)

# test dataloader
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        root='./mnist_data',
        train=False,
        download=False,
        transform=transforms.Compose([
            transforms.Resize((28,28)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])
    ),
    batch_size=FLAGS.batch_size
)

### Build Model

In [0]:
class Flatten(nn.Module):
  def forward(self, x):
    return x.view(x.size()[0], -1)

In [0]:
class model_fn(nn.Module):
    def __init__(self):
        super(model_fn, self).__init__()
        
        self.cnn = nn.Sequential(
            nn.Conv2d(1,16,kernel_size=2, padding=1, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(p=0.3),
            nn.Conv2d(16, 32, kernel_size=2, padding=1, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(p=0.3),
            nn.Conv2d(32, 32, kernel_size=2, padding=1, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(p=0.3),
            Flatten(),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(512, 10),
            nn.Softmax()
        )
    #
    
    def forward(self, x):        
        x = self.cnn(x)
        x = x.view(x.size(0), -1)
        return x
        

### Training Process

In [154]:
# Show Model Architecture
print(model_fn())

model_fn(
  (cnn): Sequential(
    (0): Conv2d(1, 16, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.3)
    (4): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout(p=0.3)
    (8): Conv2d(32, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Dropout(p=0.3)
    (12): Flatten()
    (13): ReLU()
    (14): Dropout(p=0.5)
    (15): Linear(in_features=512, out_features=10, bias=True)
    (16): Softmax()
  )
)


In [156]:
# get model
model = model_fn()
model = model.to('cuda')

# optimizer
optimizer = torch.optim.Adam(model.parameters(),lr=FLAGS.lr)
criterion = nn.BCELoss().cuda()

# train start
model.train()
for epoch in range(FLAGS.num_epoch):
  running_loss = 0.000
  for i, (inputs, labels) in enumerate(train_loader):
    model.zero_grad()
    outputs = model.cnn(inputs.cuda())
    
    labels_one_hot =  torch.eye(10)[labels]
    loss = criterion(outputs.cuda(), labels_one_hot.cuda())
    running_loss += loss
    loss.backward()
    optimizer.step()
    
  print( " ### Num. of Epochs {}  | Loss {} ### ".format(
        epoch,running_loss/float(len(train_loader.dataset)//FLAGS.batch_size) ), flush=True
  )
    

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


 ### Num. of Epochs 0  | Loss 0.10621947050094604 ### 
 ### Num. of Epochs 1  | Loss 0.05522262677550316 ### 
 ### Num. of Epochs 2  | Loss 0.04756315052509308 ### 
 ### Num. of Epochs 3  | Loss 0.043285928666591644 ### 
 ### Num. of Epochs 4  | Loss 0.04122423008084297 ### 


#### Evaluation

In [0]:
def test(model, test_loader):
  model.eval()
  correct = 0.000
  
  for data, target in test_loader:
    data, target = torch.autograd.Variable(data), torch.autograd.Variable(target)
    output = model(data.cuda())
    
    pred = output.data.max(1, keepdim=True)[1].cpu() # output label
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    
  print("Num. of Right Answers {}/{} | Accuracy in Test data {} %".format( correct,len(test_loader.dataset), 100. * float(correct) / len(test_loader.dataset)))

In [153]:
test(model, test_loader)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Num. of Right Answers 9786/10000 | Accuracy in Test data 97.86 %
